In [38]:
import getpass
import os
from credentials import *

In [39]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint= Azure_OpenAI_OB_Endpoint ,
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key= Azure_OpenAI_OB_Key,
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy",
    #streaming=True,
)

## 1 - Test

In [40]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. 
            Pour avoir du contexte , sache que ce texte est extrait du {code}"""),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")]
)

In [41]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

contenu_ancien = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
contenu_nouveau = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
llm_output = chain.invoke(
    {
        "code": "Code monétaire et financier",
        "old_version": contenu_ancien,
        "new_version": contenu_nouveau,
        "input": "I love programming.",
    }
)

In [42]:
print(llm_output)

L'analyse des modifications apportées à l'article du Code monétaire et financier révèle une suppression notable. En effet, la nouvelle version de l'article a éliminé toute mention des "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts". 

### Ancienne Version
L'ancienne version de l'article incluait explicitement les "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" parmi les entités qui doivent s'assurer que les intermédiaires en opérations de banque et en services de paiement sont immatriculés conformément à l'article L. 519-3-1. De plus, ces prestataires étaient également mentionnés dans le contexte des intermédiaires ressortissants d'un État membre de l'Union européenne ou d'un État partie à l'accord sur l'Espace économique européen exerçant en France en régime de libre prestation de services ou de libre établissement.

### Nouvelle Version
L

## 2 - Function

In [87]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def llm_legal_change_com_v1(old_version, new_version): # "Code monétaire et financier"
    

    prompt = ChatPromptTemplate.from_messages([("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. Tu n'inventeras rien et tu te baseras uniquement sur les données fournies.
            Inutile de reciter les textes fournis, tu ne retourneras que ton analyse.
            """),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "old_version": old_version,
        "new_version": new_version})
    
    return llm_output

In [88]:
llm_legal_change_com_v1(contenu_ancien, contenu_nouveau)

'L\'analyse des modifications apportées à l\'article révèle une suppression notable. Dans la nouvelle version de l\'article, les "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d\'octroi de prêts" ont été omis. \n\nCette suppression peut avoir plusieurs implications réglementaires et pratiques :\n\n1. **Réduction du champ d\'application** : La nouvelle version de l\'article réduit le champ d\'application en excluant explicitement les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d\'octroi de prêts. Cela pourrait signifier que ces prestataires ne sont plus soumis aux mêmes obligations de vérification d\'immatriculation que les autres entités mentionnées.\n\n2. **Simplification réglementaire** : En retirant cette catégorie spécifique, le législateur pourrait viser une simplification des obligations réglementaires pour les établissements de crédit, sociétés de financement, et aut

## 3 - Data prep

In [46]:
import pandas as pd
AML5_modifs= pd.read_excel("/Users/oussa/Desktop/Github_perso/3 - Legal FR Tracker/data_output/Output_AML_5.xlsx")

In [48]:
# Filter on AML text
AML5_modifs =  AML5_modifs[AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-1544")| AML5_modifs['Titre Article Modificateur'].str.contains("n°2020-115") ]

In [54]:
#check results
AML5_modifs['Titre Article Modificateur'].unique()

array(['Ordonnance n°2020-1544 du 9 décembre 2020 - art. 1',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 2',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 3',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 4',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 7',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 8',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 9',
       'Ordonnance n°2020-1544 du 9 décembre 2020 - art. 10',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 1',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 2',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 3',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 4',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 5',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 6',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 7',
       'Ordonnance n°2020-115 du 12 février 2020 - art. 8',
       'Ordonnance n°2020-115 d

In [49]:
# Keep only changed articles in their content
AML5_modifs = AML5_modifs[AML5_modifs['Compare contenu'] =="OK change"]
AML5_modifs['Compare contenu'].unique()

array(['OK change'], dtype=object)

In [50]:
# Delete KO in Old and new articles
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Ancien_Article'].str.contains("KO", na=False)]
AML5_modifs = AML5_modifs[~AML5_modifs['Contenu_Nouv_Vers_Article'].str.contains("KO", na=False)]

In [56]:
len(AML5_modifs)

54

## 5 - Apply


In [ ]:
#   AML5_modifs
#   llm_legal_change_com()

In [67]:
AML5_modifs_select = AML5_modifs.iloc[:10]

In [84]:
AML5_modifs_select['LLM_Change_Analysis_1'] = AML5_modifs_select.apply(
    lambda row: llm_legal_change_com_v1(row['Contenu_Ancien_Article'], row['Contenu_Nouv_Vers_Article']), 
    axis=1
)

In [86]:
AML5_modifs_select.to_excel("output_llm_change_analysis.xlsx")